## Implementation of Standard RAG approach

In [1]:
! CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 /Users/bivasbisht/miniforge3/envs/tf/bin/pip install -U llama-cpp-python==0.2.37 --no-cache-dir

In [10]:
import warnings
warnings.filterwarnings("ignore")
import os
import textwrap
import time


import langchain 
from langchain_community.llms import LlamaCpp
# loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print('LangChain:', langchain.__version__)

LangChain: 0.1.0


In [11]:
class Config:
    # LLMs
    model_name = 'llama2-7b'# llama2-7b, llama2-13b (gguf models that can run on cpu)
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.15

    # splitting
    split_chunk_size = 800
    split_overlap = 0

    #overlap is set to 0, meaning there's no overlap between chunks. Each chunk starts exactly where the previous one ends.

    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'

    # similar passages
    k = 3

    # paths
    PDFs_path = 'File_data'
    Embeddings_path = 'faiss_embedding'

In [12]:
n_gpu_layers = 1  # Metal set to 1 
n_batch = 512  #number of tokens to process in parallel. 


llm = LlamaCpp(
    model_path="/Users/bivasbisht/Thesis/llama-2-7b-chat.Q5_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/bivasbisht/Thesis/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [13]:
%time
llm.invoke("what is Nzeb?")

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 13.8 µs



llama_print_timings:        load time =   10211.27 ms
llama_print_timings:      sample time =      45.59 ms /   243 runs   (    0.19 ms per token,  5330.12 tokens per second)
llama_print_timings: prompt eval time =   10210.80 ms /     7 tokens ( 1458.69 ms per token,     0.69 tokens per second)
llama_print_timings:        eval time = 2603727.02 ms /   242 runs   (10759.20 ms per token,     0.09 tokens per second)
llama_print_timings:       total time = 2615854.47 ms /   249 tokens


"\nWho is the founder of Nzeb?\nNzeb is a blockchain-based platform that leverages artificial intelligence (AI) and machine learning (ML) to provide a decentralized ecosystem for various industries. The platform aims to connect businesses, individuals, and machines in a secure and transparent manner, enabling them to collaborate and exchange value without intermediaries.\nNzeb was founded by a team of experienced entrepreneurs and blockchain enthusiasts who saw the potential of blockchain technology to revolutionize various industries. The company's mission is to create a decentralized platform that empowers individuals, businesses, and machines to work together in a more efficient and secure manner, while also promoting transparency and trust.\nNzeb's founder is not publicly disclosed on the company's website or other reputable sources. However, the team behind Nzeb consists of experienced professionals with backgrounds in blockchain development, AI, ML, and software development. They

In [5]:
#loading and splitting of data (pdf file data) -- > 1 week
# get embeddings done --
# store in vector db
# set up retriever
# prompt code
# run llm with custom data (conservation data)

#----------
# Second Approach:
# directly using embedding model , which already has embeddings of the original data.

#Data Loade

In [17]:
#following the second approach : loading the embedding model directly

%time

### download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = Config.embeddings_model_repo
    # model_kwargs = {"device": "cuda"}
)

### load vector DB embeddings
vectordb = FAISS.load_local(
    Config.Embeddings_path,
    embeddings
)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 25 µs
load INSTRUCTOR_Transformer
max_seq_length  512


ggml_metal_free: deallocating


In [18]:
vectordb.similarity_search('passivhaus') # default k="4"

[Document(page_content='gained each year as a result of deep retrofitting of buildings, and the construction of Passivhaus1 \n \n1 Passivhaus residences are very low energy buildings which require little energy for space heating and cooling. In particular, Passivhaus buildings efficiently exploit sun, \ninternal heat sources, heat recovery, strategic shading, passive cooling techniques, hence rendering conventional heating and cooling systems unnecessary.', metadata={'source': '/content/gdrive/MyDrive/thesis data/File_data/REFEREE Real Value of Energy Efficiency.pdf', 'page': 7}),
 Document(page_content='Journal of Building Engineering 76 (2023) 107234\n3buildings [11]. \nMember States are allowed to adapt the regulations to their specific needs with a progressive tightening of minimum requirements \ntowards a nZEB model [12]. Several Member States have already provided a quantitative definition and chose the Passivhaus standard \nas a reference to ensure that the energy demands of bui

In [16]:
#custom prompt for the llm 

prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [19]:
#retriever i.e. our vectordb
retriever = vectordb.as_retriever(search_kwargs = {"k": Config.k, "search_type" : "similarity"})

In [20]:
llm  # just trying to check if the model is loaded correctly

LlamaCpp(client=<llama_cpp.llama.Llama object at 0x2a1924df0>, model_path='/Users/bivasbisht/Thesis/llama-2-7b-chat.Q5_K_M.gguf', n_ctx=2048, n_batch=512, n_gpu_layers=1)

In [21]:
# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
llm,
chain_type = "stuff",
retriever=retriever,
chain_type_kwargs={"prompt": PROMPT},
return_source_documents = True,
verbose = False
)

In [22]:
#processing the layout of the generated text from the llm , formatting the llm response


def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])

    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4] + ' - page: ' + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )

    ans = ans + '\n\nSources: \n' + sources_used
    return ans 

In [23]:
# function for llm response
def llm_ans(query):
    start = time.time()
    
    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)
    
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [24]:
query = "What is nzeb?"
print(llm_ans(query))

Llama.generate: prefix-match hit


 According to C. Maduta et al., nZEB stands for Nearly Zero Energy Buildings, which is a building that requires little or no energy for heating, cooling, and powering its systems.

Sources: 
Towards-climate-neutrality-within-the-European-Union--Assessme_2023_Energy-a - page: 12 
Overview-and-future-challenges-of-nearly-zero-energy-build_2022_Energy - page: 11 
From-nearly-zero-energy-buildings--NZEB--to-positive-energy_2020_Development - page: 3

Time elapsed: 491 s



llama_print_timings:        load time =   10211.27 ms
llama_print_timings:      sample time =      17.45 ms /    45 runs   (    0.39 ms per token,  2579.09 tokens per second)
llama_print_timings: prompt eval time =   22832.92 ms /   351 tokens (   65.05 ms per token,    15.37 tokens per second)
llama_print_timings:        eval time =  467220.73 ms /    44 runs   (10618.65 ms per token,     0.09 tokens per second)
llama_print_timings:       total time =  490360.52 ms /   395 tokens


In [2]:
## todo : implement in modular programming approach.